In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
#Load the training data
train_data = pd.read_csv('train_data.csv')
X_train = train_data.drop('label', axis=1).values
Y_train = train_data['label'].values

In [3]:
#Determine the number of classes dynamically
NumClasses = train_data['label'].nunique()

In [4]:
#Ensure the number of features matches the input shape expected by the model
input_shape=X_train.shape[1]

In [5]:
#Load the testing data
test_data = pd.read_csv('test_data.csv')
X_test = test_data.values

In [6]:
#Normalize pixel value to be between 0 and 1
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [7]:
if X_test.shape[1] != input_shape:
    X_test = X_test.reshape((X_test.shape[0], -1))

In [8]:
# Check the current shape and size of X_test
print("X_test shape before reshaping:", X_test.shape)
print("X_test size before reshaping:", X_test.size)


X_test shape before reshaping: (10000, 3073)
X_test size before reshaping: 30730000


In [9]:
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)

In [10]:
#Split the training data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [11]:
# Reshape X_train and X_val
X_train = X_train.reshape(-1, 32, 32, 3)
X_val = X_val.reshape(-1, 32, 32, 3)

In [12]:
# Convert integer labels to one-hot encoded labels
Y_train_one_hot = to_categorical(Y_train, num_classes=NumClasses)
Y_val_one_hot = to_categorical(Y_val, num_classes=NumClasses)

In [13]:
#Model definition
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 32)          18464     
                                                                 
 flatten (Flatten)           (None, 512)              

In [16]:
#model.compile(optimizer= 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Training the model
epochs = 10
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val))


Epoch 1/10
1250/1250 [==============================] - 39s 31ms/step - loss: 0.9209 - accuracy: 0.6760 - val_loss: 0.9675 - val_accuracy: 0.6655
Epoch 2/10
1250/1250 [==============================] - 44s 35ms/step - loss: 0.8729 - accuracy: 0.6928 - val_loss: 0.9351 - val_accuracy: 0.6753
Epoch 3/10
1250/1250 [==============================] - 43s 34ms/step - loss: 0.8259 - accuracy: 0.7121 - val_loss: 0.9339 - val_accuracy: 0.6792
Epoch 4/10
1250/1250 [==============================] - 36s 29ms/step - loss: 0.7932 - accuracy: 0.7210 - val_loss: 0.9220 - val_accuracy: 0.6811
Epoch 5/10
1250/1250 [==============================] - 37s 29ms/step - loss: 0.7581 - accuracy: 0.7329 - val_loss: 0.9345 - val_accuracy: 0.6824
Epoch 6/10
1250/1250 [==============================] - 36s 28ms/step - loss: 0.7239 - accuracy: 0.7436 - val_loss: 0.9998 - val_accuracy: 0.6657
Epoch 7/10
1250/1250 [==============================] - 36s 28ms/step - loss: 0.6943 - accuracy: 0.7570 - val_loss: 0.9421 -

In [ ]:
test_loss, test_acc = model.evaluate(X_val, Y_val, verbose = 2)
print('\nTest accuracy: ', test_acc)
print('\nTest Loss: ', test_loss)